In [0]:
!pip install gensim --user

In [0]:
from google.colab import files
from google.colab import drive
import argparse
import gzip, os, csv
import numpy as np
import random
import time
import io
import gensim

## create data structure for knowledge graph
def addTriple(net, source, target, edge):
    if source in net:
        if  target in net[source]:
            net[source][target].add(edge)
        else:
            net[source][target]= set([edge])
    else:
        net[source]={}
        net[source][target] =set([edge])
            
def getLinks(net, source):
    if source not in net:
        return {}
    return net[source]

# Generate paths (entity->relation->entity) by radom walks
def randomWalkUniform(triples, startNode, max_depth=5):
    next_node =startNode
    path = str(startNode)+'->'
    for i in range(max_depth):
        neighs = getLinks(triples,next_node)
        #print (neighs)
        if len(neighs) == 0: break
        weights = []
        queue = []
        for neigh in neighs:
            for edge in neighs[neigh]:
                queue.append((edge,neigh))
        edge, next_node = random.choice(queue)
        path = path +str(edge)+'->'
        path = path +str(next_node)+'->'
    path =path.split('->')
    return path

# Build the knowledge graph structure
def preprocess(fname):
    triples = {}

    ent_counter = 0
    rel_counter = 0
    train_counter = 0

    print (fname)
    #gzfile= gzip.open(fname, mode='rt')

    for line in csv.reader(open(fname), delimiter='\t', quotechar='"'):
        #print (line)
        h = line[0]
        r = line[1]
        t = line[2]
        
        train_counter +=1

        addTriple(triples, h, t, r)
        train_counter+=1
    print ('Triple:',train_counter)
    return triples

# Randon walk functions
def randomNWalkUniform(triples, n, walks, path_depth):
    path=[]
    for k in range(walks):
        walk = randomWalkUniform(triples, n, path_depth)
        path.append(walk)
    return path

In [18]:
#file = 'test.nt'
#file = files.upload()
#file = io.BytesIO('test.nt')

drive.mount('/content/drive')



#triples = preprocess(/content/drive/)
#print(triples)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
os.listdir('/content/drive/My Drive/Data/')

['rep_out.nt', 'kg4gcr.nt', 'statements.tsv']

In [19]:
triples = preprocess('/content/drive/My Drive/Data/statements.tsv')
#print(triples)
entities = list(triples.keys())
vocabulary = entities
print (len(vocabulary))
vocabulary[:10]

/content/drive/My Drive/Data/statements.tsv
Triple: 6087512
8885


['<http://kg4gcr.com/repository/LisaMaatje%2FARS>',
 '<http://kg4gcr.com/contributor/LisaMaatje>',
 '<http://kg4gcr.com/contributor/apyspo>',
 '<http://kg4gcr.com/repository/ytdl-org%2Fyoutube-dl>',
 '<http://kg4gcr.com/contributor/dstftw>',
 '<http://kg4gcr.com/contributor/phihag>',
 '<http://kg4gcr.com/contributor/remitamine>',
 '<http://kg4gcr.com/contributor/jaimeMF>',
 '<http://kg4gcr.com/contributor/yan12125>',
 '<http://kg4gcr.com/contributor/rg3>']

In [20]:
# do randon walks
walks = 5000
path_depth = 5

start_time =time.time()
sentences =[]
for word in vocabulary:
    sentences.extend( randomNWalkUniform(triples, word, walks, path_depth) )
elapsed_time = time.time() - start_time
print ('Time elapsed to generate features:',time.strftime("%H:%M:%S", time.gmtime(elapsed_time)))

Time elapsed to generate features: 00:44:06


In [21]:
model1 = gensim.models.Word2Vec(size=100, workers=8, window=5, sg=1, negative=15, iter=5)
model1.build_vocab(sentences)
corpus_count = model1.corpus_count
model1.train(sentences, total_examples = corpus_count, epochs = 5)


(478792130, 888889230)

In [35]:
#model1.save("word2vec.model")
#model1.wv.most_similar(positive=['<http://kg4gcr.com/contributor/LisaMaatje>', '<https://www.wikidata.org/wiki/Property:P1327>'])
model1.most_similar(positive=['<http://kg4gcr.com/contributor/apyspo>', '<http://kg4gcr.com/contributor/rcelebi>'], negative=['<http://kg4gcr.com/contributor/LisaMaatje>'])
#model1.most_similar(positive=['<http://kg4gcr.com/contributor/rcelebi>', '<http://kg4gcr.com/contributor/apyspo>'], negative=['<http://kg4gcr.com/contributor/vemonet>'])


#model1.most_similar(positive=['<http://kg4gcr.com/contributor/LisaMaatje>', '<https://www.wikidata.org/wiki/Property:P1327>'], negative=['<http://kg4gcr.com/contributor/apyspo>'])
#model1.most_similar(positive=['<http://kg4gcr.com/contributor/rcelebi>', '<https://www.wikidata.org/wiki/Property:P1327>'], negative=['<http://kg4gcr.com/contributor/vemonet>'])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('<http://kg4gcr.com/contributor/vemonet>', 0.966220498085022),
 ('<http://kg4gcr.com/contributor/ammar257ammar>', 0.9582265019416809),
 ('<http://kg4gcr.com/contributor/swd543>', 0.9390175342559814),
 ('<http://kg4gcr.com/contributor/yunfei-shi>', 0.6450794339179993),
 ('<http://kg4gcr.com/contributor/sty945>', 0.6445444226264954),
 ('<http://kg4gcr.com/contributor/BrikerMan>', 0.626862108707428),
 ('https://github.com/jamesstidard', 0.6210993528366089),
 ('https://github.com/alexallah', 0.6204466819763184),
 ('https://github.com/erinxocon', 0.6189807653427124),
 ('https://github.com/reinout', 0.6188479065895081)]

-0.07008165
